In [ ]:
import math
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
import time
import os

In [15]:
ZOOM = 18
IMG_SIZE = (224, 224)

API_KEY = os.getenv("SATELLITE_API_KEY")

In [ ]:
os.makedirs("../Data/images/train", exist_ok=True)
os.makedirs("../Data/images/test", exist_ok=True)

In [ ]:
train_df = pd.read_excel("../Data/train(1).xlsx")
test_df = pd.read_excel("../Data/test2.xlsx")

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,7,1810,0,1961,0,98055,47.4362,-122.187,1660,9240
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,7,1600,0,1992,0,98031,47.4034,-122.187,1720,3605
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,8,1720,0,1994,0,98003,47.2704,-122.313,1870,7455
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,7,1150,90,2009,0,98027,47.5321,-122.073,1240,750
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,7,1280,0,1994,0,98042,47.3715,-122.074,1590,8071


In [ ]:
def latlon_to_tile(lat, lon, zoom):
    lat_rad = math.radians(lat)
    n = 2 ** zoom

    x = int((lon + 180.0) / 360.0 * n)
    y = int(
        (1.0 - math.log(math.tan(lat_rad) + 1 / math.cos(lat_rad)) / math.pi)
        / 2.0 * n
    )

    return x, y

In [ ]:
def download_images(df, img_dir, max_images=None):
    count = 0

    for _, row in df.iterrows():
        img_id = row["id"]
        lat = row["lat"]
        lon = row["long"]

        img_path = f"{img_dir}/{img_id}.png"

        if os.path.exists(img_path):
            continue

        x, y = latlon_to_tile(lat, lon, ZOOM)

        url = f"https://api.maptiler.com/tiles/satellite-v2/{ZOOM}/{x}/{y}.jpg?key=**************fem"

        try:
            response = requests.get(url, timeout=10)
            if response.status_code != 200:
                print(f"Failed for id {img_id}")
                continue

            img = Image.open(BytesIO(response.content))
            img = img.resize(IMG_SIZE)
            img.save(img_path)

            count += 1
            print(f"Saved {img_path}")

            time.sleep(0.2)

            if max_images and count >= max_images:
                break

        except Exception as e:
            print(f"Error for id {img_id}: {e}")

In [19]:
# download_images(train_df, TRAIN_IMG_DIR)

In [ ]:

# download_images(test_df, TEST_IMG_DIR)